In [17]:
# Clone PyFM
!git clone https://github.com/RobinMagnet/pyFM.git pyfm_repo
!mv pyfm_repo/pyFM .
!rm -rf pyfm_repo

Cloning into 'pyfm_repo'...
remote: Enumerating objects: 512, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 512 (delta 59), reused 56 (delta 50), pack-reused 436
Receiving objects: 100% (512/512), 942.38 KiB | 10.83 MiB/s, done.
Resolving deltas: 100% (324/324), done.
mv: cannot move 'pyfm_repo/pyFM' to './pyFM': Directory not empty



# 1 - Imports and defining functions

In [2]:
import numpy as np

from pyFM.mesh import TriMesh
from pyFM.functional import FunctionalMapping
import trimesh
from scipy.spatial import distance
import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

def plot_correspondence_lines(mesh1, mesh2, mesh1_points, mesh2_points):
    def plot_mesh_and_lines(mesh, points):
        p = mp.plot(mesh1_points, None, c=visu(points))
        p.add_mesh(mesh.vertices, mesh.faces, c=visu(mesh.vertices))
        p.add_lines(mesh1_points, mesh2_points)
    plot_mesh_and_lines(mesh1, mesh2_points)
    plot_mesh_and_lines(mesh2, mesh1_points)

In [42]:

toy = ['blue', 'yellow'][0]

if toy == 'yellow':
    prefix = 'yellow_'
    dir_name = 'YellowToy01'
else:
    prefix = ''
    dir_name = 'BlueToy02'
for num in range(3,9):
    mesh2 = TriMesh(f'data/dataset/{dir_name}/{prefix}push_toy_1_70000.obj')
    mesh1 = TriMesh(f'data/dataset/{dir_name}/{prefix}push_toy_{num}_70000.obj')
    print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
        f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')
    process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt(f'data/{toy}_toy_1-{num}_landmarks_deformed_and_non.txt',dtype=int),  # loading 20 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
    }

    model = FunctionalMapping(mesh1,mesh2)
    model.preprocess(**process_params,verbose=True);
    fit_params = {
    'w_descr': 1e0,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
    }



    model.fit(**fit_params, verbose=True)
    p2p_21 = model.get_p2p(n_jobs=1)
    cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
    model.icp_refine(verbose=True)
    p2p_21_icp = model.get_p2p()
    cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
    random_indexes=np.random.permutation(p2p_21_icp.shape[0])[:2048]
    p2p_21_icp_2048=p2p_21_icp[random_indexes]
    np.save(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/icp/deformed_{num}_correspondences_zoom_2048.npy',mesh1.vertlist[p2p_21_icp_2048])
    np.save(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/icp/non_deformed_{num}_correspondences_zoom_2048.npy',mesh2.vertlist[random_indexes])
    cloud_1_2048 = trimesh.PointCloud(mesh1.vertlist[p2p_21_icp_2048])
    cloud_1_2048.export(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/icp/deformed_{num}_correspondences_zoom_2048.ply')
    cloud_2_2048 = trimesh.PointCloud(mesh2.vertlist[random_indexes])
    cloud_2_2048.export(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/icp/non_deformed_{num}_correspondences_zoom_2048.ply')
    model.change_FM_type('classic') # We refine the first computed map, not the icp-refined one
    model.zoomout_refine(nit=15, step = 1, verbose=True)
    #print(model.FM.shape)
    p2p_21_zo = model.get_p2p()
    cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
    random_indexes=np.random.permutation(p2p_21_zo.shape[0])[:2048]
    p2p_21_zo_2048=p2p_21_zo[random_indexes]
    np.save(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/zoomout/deformed_{num}_correspondences_zoom_2048.npy',mesh1.vertlist[p2p_21_zo_2048])
    np.save(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/zoomout/non_deformed_{num}_correspondences_zoom_2048.npy',mesh2.vertlist[random_indexes])
    cloud_1_2048 = trimesh.PointCloud(mesh1.vertlist[p2p_21_zo_2048])
    cloud_1_2048.export(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/zoomout/deformed_{num}_correspondences_zoom_2048.ply')
    cloud_2_2048 = trimesh.PointCloud(mesh2.vertlist[random_indexes])
    cloud_2_2048.export(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/zoomout/non_deformed_{num}_correspondences_zoom_2048.ply')


Mesh 1 : 36821 vertices, 70000 faces
Mesh 2 : 35002 vertices, 70000 faces

Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 12.71 s
Computing 200 eigenvectors
	Done in 13.71 s

Computing descriptors
	Normalizing descriptors

	420 out of 2100 possible descriptors kept
Computing commutativity operators
	Scaling LBO commutativity weight by 1.4e-11

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 420 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 17, nit : 15, warnflag : 0
	Done in 0.71 seconds


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

Mesh 1 : 36956 vertices, 69999 faces
Mesh 2 : 35002 vertices, 70000 faces

Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 11.47 s
Computing 200 eigenvectors
	Done in 13.24 s

Computing descriptors
	Normalizing descriptors

	420 out of 2100 possible descriptors kept
Computing commutativity operators
	Scaling LBO commutativity weight by 1.4e-11

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 420 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 18, nit : 15, warnflag : 0
	Done in 0.73 seconds


  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

Mesh 1 : 35818 vertices, 70000 faces
Mesh 2 : 35002 vertices, 70000 faces

Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 15.40 s
Computing 200 eigenvectors
	Done in 20.72 s

Computing descriptors
	Normalizing descriptors

	420 out of 2100 possible descriptors kept
Computing commutativity operators
	Scaling LBO commutativity weight by 2.1e-11

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 420 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 16, nit : 14, warnflag : 0
	Done in 1.24 seconds


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

Select the shape

In [33]:
toy = ['blue', 'yellow'][0]
num = 2

if toy == 'yellow':
    prefix = 'yellow_'
    dir_name = 'YellowToy01'
else:
    prefix = ''
    dir_name = 'BlueToy02'

# 2 - Computing the functional map

**Loading data**

In [34]:
mesh2 = TriMesh(f'data/dataset/{dir_name}/{prefix}push_toy_1_70000.obj')
mesh1 = TriMesh(f'data/dataset/{dir_name}/{prefix}push_toy_{num}_70000.obj')
print(f'Mesh 1 : {mesh1.n_vertices:4d} vertices, {mesh1.n_faces:5d} faces\n'
      f'Mesh 2 : {mesh2.n_vertices:4d} vertices, {mesh2.n_faces:5d} faces')

double_plot(mesh1,mesh2)

Mesh 1 : 36990 vertices, 70000 faces
Mesh 2 : 35002 vertices, 70000 faces


In [5]:
"""from scipy.spatial import distance
import numpy as np
rand_idx=np.random.permutation(mesh1.vertlist.shape[0])[:20]
print(rand_idx)
euc_dist=distance.cdist(mesh1.vertlist[rand_idx],mesh2.vertlist)"""

[22154 11507 26982 28103 14412  3720 25905 14933 28262  4434 14984 10799
 21847 18331 21147 15834 28495  3802  2418 34554]


In [6]:
#temp=np.argmin(euc_dist, axis=1)

In [23]:
# Manually change the landmark file name
"""with open("data/yellow_toy_1-"+str(num)+"_landmarks_deformed_and_non.txt", 'w+') as fp:
    
    for i in range(temp.shape[0]):
        fp.write(str(rand_idx[i])+" "+str(temp[i])+"\n")

with open("data/yellow_toy_1-"+str(num)+"_landmarks_non_and_deformed.txt", 'w+') as fp:
    
    for i in range(temp.shape[0]):
        fp.write(str(temp[i])+" "+str(rand_idx[i])+"\n")
"""

'with open("data/yellow_toy_1-"+str(num)+"_landmarks_deformed_and_non.txt", \'w+\') as fp:\n    \n    for i in range(temp.shape[0]):\n        fp.write(str(rand_idx[i])+" "+str(temp[i])+"\n")\n\nwith open("data/yellow_toy_1-"+str(num)+"_landmarks_non_and_deformed.txt", \'w+\') as fp:\n    \n    for i in range(temp.shape[0]):\n        fp.write(str(temp[i])+" "+str(rand_idx[i])+"\n")\n'

**Computing descriptors**

In [35]:
process_params = {
    'n_ev': (35,35),  # Number of eigenvalues on source and Target
    'landmarks': np.loadtxt(f'data/{toy}_toy_1-{num}_landmarks_deformed_and_non.txt',dtype=int),  # loading 20 landmarks
    'subsample_step': 5,  # In order not to use too many descriptors
    'descr_type': 'WKS',  # WKS or HKS
}

model = FunctionalMapping(mesh1,mesh2)
model.preprocess(**process_params,verbose=True);


Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 12.47 s
Computing 200 eigenvectors
	Done in 13.58 s

Computing descriptors
	Normalizing descriptors

	420 out of 2100 possible descriptors kept


**Fitting the model**

$\renewcommand{\RR}{\mathbb{R}}$
$\renewcommand{\Ss}{\mathcal{S}}$
$\renewcommand{\uargmin}[1]{\underset{#1}{\text{argmin}}\;}$
$\renewcommand{\uargmax}[1]{\underset{#1}{\text{argmax}}\;}$
$\def\*#1{\mathbf{#1}}$

In pyFM, we always consider functional maps $\*C:\Ss_1\to\Ss_2$ and pointwise maps $T:\Ss_2\to\Ss_1$ going in opposite directions, with $\*C$ always going from shape 1 to shape 2 !

Optimization problem is
\begin{equation}
\uargmin{\*C\in\RR^{k_2\times k_1}} w_{descr}\|\*C\*A - \*B\|^2 + w_{lap}\|\*C\Delta_1 - \Delta_2\*C\|^2 + w_{\text{d- comm}}\sum_i \|\*C\Gamma_1^i - \Gamma_2^i\*C\|^2 + w_{\text{orient}}\sum_i \|\*C\Lambda_1^i - \Lambda_2^i\*C\|^2
\end{equation}

with $\Gamma_1^i$ and $\Gamma_2^i$ [multipliative operators](http://www.lix.polytechnique.fr/~maks/papers/fundescEG17.pdf) associated to the $i$-th descriptors, $\Lambda_1^i$ and $\Lambda_2^i$ [orientation preserving operators](https://arxiv.org/abs/1806.04455) associated to the $i$-th descriptors

In [36]:
fit_params = {
    'w_descr': 1e0,
    'w_lap': 1e-2,
    'w_dcomm': 1e-1,
    'w_orient': 0
}



model.fit(**fit_params, verbose=True)

Computing commutativity operators
	Scaling LBO commutativity weight by 1.5e-11

Optimization :
	35 Ev on source - 35 Ev on Target
	Using 420 Descriptors
	Hyperparameters :
		Descriptors preservation :1.0e+00
		Descriptors commutativity :1.0e-01
		Laplacian commutativity :1.0e-02
		Orientation preservation :0.0e+00

	Task : CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH, funcall : 19, nit : 16, warnflag : 0
	Done in 0.75 seconds


**Visualizing the associated point to point map**

In [37]:
p2p_21 = model.get_p2p(n_jobs=1)
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
double_plot(mesh1,mesh2,cmap1,cmap2)

# 3 - Refining the Functional Map
```model.FM``` returns the current state of functional map. One can change which one is returned by using ```model.change_FM_type(FM_type)```, as one can see below. 

**ICP**

In [38]:
model.icp_refine(verbose=True)
p2p_21_icp = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_icp]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/10 [00:00<?, ?it/s]

In [39]:
random_indexes=np.random.permutation(p2p_21_icp.shape[0])[:2048]
p2p_21_icp_2048=p2p_21_icp[random_indexes]
np.save(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/icp/deformed_{num}_correspondences_zoom_2048.npy',mesh1.vertlist[p2p_21_icp_2048])
np.save(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/icp/non_deformed_{num}_correspondences_zoom_2048.npy',mesh2.vertlist[random_indexes])
cloud_1_2048 = trimesh.PointCloud(mesh1.vertlist[p2p_21_icp_2048])
cloud_1_2048.export(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/icp/deformed_{num}_correspondences_zoom_2048.ply')
cloud_2_2048 = trimesh.PointCloud(mesh2.vertlist[random_indexes])
cloud_2_2048.export(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/icp/non_deformed_{num}_correspondences_zoom_2048.ply')


b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 2048\nproperty float x\nproperty float y\nproperty float z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nproperty uchar alpha\nend_header\n\xb4\xe3\xa6=\x1a\x8b&<*R!>\x00\x00\x00\x00\x8fU\xca\xbc^08\xbc\t\xc25>\x00\x00\x00\x00\x98L\x15\xbd\xe4\x86\xdf\xbc\xc2P\'>\x00\x00\x00\x00m\x8c\x1d\xba\xc0<$=7\x8e8>\x00\x00\x00\x00\xd5\xcc\xda\xbc\xad2S=\xa5\xa1&>\x00\x00\x00\x00\xbb*\xd0<\xec\xc09\xbd|\xd29>\x00\x00\x00\x00\xdb\xf8S=\x82\xa9\x86=x{0>\x00\x00\x00\x00\x80\x0c\x1d=V*\xa8\xbbq\x93A>\x00\x00\x00\x00\xe36Z=Y\xa54=\xb8\x938>\x00\x00\x00\x00\xaa\x10\x0f;\'1H\xbd\'/">\x00\x00\x00\x00\xc21\x0b=\xaa)\xc9;\x0c\x93\x19>\x00\x00\x00\x00b\xd76=|H\xf8\xbc~:\x1e>\x00\x00\x00\x00\xf8\xfb\x85=U\xda\x82=1\x97$>\x00\x00\x00\x00\xd2\xe5\r=.\xad\x06;\x13\x99\x19>\x00\x00\x00\x00W\xce\xde\xbc\x1fh\x85<Y\xa54>\x00\x00\x00\x00\xa1\xf5p=\xb2\x85\x80\xbd\xfa&->\x00\x00\x00\x00\x0f\x0b5<\x12\x

**Zoomout**

In [40]:
model.change_FM_type('classic') # We refine the first computed map, not the icp-refined one
model.zoomout_refine(nit=15, step = 1, verbose=True)
print(model.FM.shape)
p2p_21_zo = model.get_p2p()
cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21_zo]
double_plot(mesh1,mesh2,cmap1,cmap2)

  0%|          | 0/15 [00:00<?, ?it/s]

(50, 50)


### Save 2048 or 35000 correspondences

2048 correspondences

In [41]:
random_indexes=np.random.permutation(p2p_21_zo.shape[0])[:2048]
p2p_21_zo_2048=p2p_21_zo[random_indexes]
np.save(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/zoomout/deformed_{num}_correspondences_zoom_2048.npy',mesh1.vertlist[p2p_21_zo_2048])
np.save(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/zoomout/non_deformed_{num}_correspondences_zoom_2048.npy',mesh2.vertlist[random_indexes])
cloud_1_2048 = trimesh.PointCloud(mesh1.vertlist[p2p_21_zo_2048])
cloud_1_2048.export(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/zoomout/deformed_{num}_correspondences_zoom_2048.ply')
cloud_2_2048 = trimesh.PointCloud(mesh2.vertlist[random_indexes])
cloud_2_2048.export(f'NonDeformed-to-Deformed-Correspondences/{dir_name}/zoomout/non_deformed_{num}_correspondences_zoom_2048.ply')


b'ply\nformat binary_little_endian 1.0\ncomment https://github.com/mikedh/trimesh\nelement vertex 2048\nproperty float x\nproperty float y\nproperty float z\nproperty uchar red\nproperty uchar green\nproperty uchar blue\nproperty uchar alpha\nend_header\n`\xcc\x16\xbcb\xbcf<m =>\x00\x00\x00\x00\xe4\xf4\xf5\xbb\tmy=\xaef->\x00\x00\x00\x00\x00\xa8\xe2\xbb\xf2\xb2&\xbd\x07)8>\x00\x00\x00\x00?\xe0\x01\xbd9\xb7I=\xc9q\'>\x00\x00\x00\x00K?a\xbb\xa5\x12^\xbd/\xdb.>\x00\x00\x00\x00\xe0\xa0=\xbcYQC\xbd\x85x$>\x00\x00\x00\x00\x84f\xd7\xbc\xad0\xfd<\r\xe3\x1e>\x00\x00\x00\x00\xfa^\xc3=\x11mG<Gs4>\x00\x00\x00\x00+Q\xf6<\xa2\xd4\xde\xbc\xf5\xf5\x1c>\x00\x00\x00\x00\x84\x7f\x91\xbc5\x99\xf1<\x91a5>\x00\x00\x00\x00`\xc9U=JE\x83\xbd[B.>\x00\x00\x00\x00\xe5D\xbb<\x08\xb0\x88=\x0c\x04!>\x00\x00\x00\x00q9\x1e= b\x83\xbc\x10\x94\x1b>\x00\x00\x00\x00\tT\x7f:#Ng=\xfdl4>\x00\x00\x00\x00\x8fS\x94=d\x94\xe7<(H<>\x00\x00\x00\x00\x00\x8f\xa8;\xcd\xcav<U3\x1b>\x00\x00\x00\x00\xce\xe1\x1a=0\xa1\xa2=\xad\xbe*>\x00\

35000 correspondences

In [53]:
"""
p2p_21_zo_35000=p2p_21_zo[:35000]
print(mesh1.vertlist[p2p_21_zo_35000].shape,mesh2.vertlist[:35000].shape)
np.save(f'2048 and 35000 Correspondences/{dir_name}/deformed_{num}_correspondences_zoom_35000.npy',mesh1.vertlist[p2p_21_zo_35000])
np.save(f'2048 and 35000 Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoom_35000.npy',mesh2.vertlist[:35000])
cloud_1 = trimesh.PointCloud(mesh1.vertlist[p2p_21_zo_35000])
cloud_1.export(f'2048 and 35000 Correspondences/{dir_name}/deformed_{num}_correspondences_zoom_35000.ply')
cloud_2 = trimesh.PointCloud(mesh2.vertlist[:35000])
cloud_2.export(f'2048 and 35000 Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoom_35000.ply')
euc_dist=distance.cdist(cloud_2.vertices,cloud_1.vertices)
min_idx=np.argmin(euc_dist, axis=1)
min_vals=np.min(euc_dist,axis=1)
deformation_dist=np.max(min_vals)
deformation_idx=np.argmax(min_vals)
f=open(f"2048 and 35000 Correspondences/{dir_name}/{toy}_deformation_1_{num}_35000.txt","w")
f.write("1 "+ str(num)+"\n")
f.write(str(cloud_2.vertices[deformation_idx])+" "+str(cloud_1.vertices[min_idx[deformation_idx]])+"\n")
f.write(str(deformation_idx)+" "+str(min_idx[deformation_idx])+"\n")
f.write(str(cloud_1.vertices[min_idx[deformation_idx]]-cloud_2.vertices[deformation_idx])+"\n")
f.write(str(deformation_dist))
f.close()
cloud_1_2048=np.load(f'2048 and 35000 Correspondences/{dir_name}/deformed_{num}_correspondences_zoom_35000.npy')
cloud_2_2048=np.load(f'2048 and 35000 Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoom_35000.npy')

distances=np.linalg.norm(cloud_2_2048-cloud_1_2048,ord=2, axis=1)
max_val=np.max(distances,axis=0)
max_idx=np.argmax(distances,axis=0)
f=open(f"2048 and 35000 Correspondences/{dir_name}/{toy}_deformation_1_{num}_35000.txt","w")
f.write("1 "+ str(num)+"\n")
f.write(str(max_idx)+"\n")
f.write(str(max_val))
f.close()"""    

### Finding bijective Correspondences

In [ ]:
"""a=np.load(f'Deformed-to-Non-deformed Correspondences/{dir_name}/deformed_{num}_correspondences_icp.npy')
b=np.load(f'Non-deformed-to-Deformed Correspondences/{dir_name}/deformed_{num}_correspondences_icp.npy')

c=np.load(f'Deformed-to-Non-deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_icp.npy')
d=np.load(f'Non-deformed-to-Deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_icp.npy')

deformed_num_icp_deformed_to_non_blue=a.tolist()
deformed_num_icp_non_to_deformed_blue=b.tolist()
nondeformed_num_icp_deformed_to_non_blue=c.tolist()
nondeformed_num_icp_non_to_deformed_blue=d.tolist()
list_bijective_deformed_icp_blue=[]
list_bijective_nondeformed_icp_blue=[]

for i,row in enumerate(deformed_num_icp_deformed_to_non_blue):
    try:
        id=deformed_num_icp_non_to_deformed_blue.index(row)
        if nondeformed_num_icp_deformed_to_non_blue[i]==nondeformed_num_icp_non_to_deformed_blue[id]:
            list_bijective_deformed_icp_blue.append(row)
            list_bijective_nondeformed_icp_blue.append(nondeformed_num_icp_deformed_to_non_blue[i])
    except:
        continue
print(len(list_bijective_deformed_icp_blue),len(list_bijective_nondeformed_icp_blue))
np.save(f'Bijective Correspondences/{dir_name}/deformed_1-{num}_bijective_icp.npy',np.array(list_bijective_deformed_icp_blue))
"""for row in nondeformed_num_icp_deformed_to_non_blue:
    if row in nondeformed_num_icp_non_to_deformed_blue:
        list_bijective_nondeformed_icp_blue.append(row)"""
np.save(f'Bijective Correspondences/{dir_name}/nondeformed_1-{num}_bijective_icp.npy',np.array(list_bijective_nondeformed_icp_blue))

e=np.load(f'Deformed-to-Non-deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoomed.npy')
f=np.load(f'Non-deformed-to-Deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoomed.npy')

g=np.load(f'Deformed-to-Non-deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoomed.npy')
h=np.load(f'Non-deformed-to-Deformed Correspondences/{dir_name}/non_deformed_{num}_correspondences_zoomed.npy')

deformed_num_zoomed_deformed_to_non_blue=a.tolist()
deformed_num_zoomed_non_to_deformed_blue=b.tolist()
nondeformed_num_zoomed_deformed_to_non_blue=c.tolist()
nondeformed_num_zoomed_non_to_deformed_blue=d.tolist()
list_bijective_deformed_zoomed_blue=[]
list_bijective_nondeformed_zoomed_blue=[]

for i,row in enumerate(deformed_num_zoomed_deformed_to_non_blue):
    try:
        id=deformed_num_zoomed_non_to_deformed_blue.index(row)
        if nondeformed_num_zoomed_deformed_to_non_blue[i]==nondeformed_num_zoomed_non_to_deformed_blue[id]:
            list_bijective_deformed_zoomed_blue.append(row)
            list_bijective_nondeformed_zoomed_blue.append(nondeformed_num_zoomed_deformed_to_non_blue[i])
            #print(id,i,row,nondeformed_num_zoomed_deformed_to_non_blue[i],nondeformed_num_zoomed_non_to_deformed_blue[id])
    except:
        continue
print(len(list_bijective_deformed_zoomed_blue),len(list_bijective_nondeformed_zoomed_blue))
np.save(f'Bijective Correspondences/{dir_name}/deformed_1-{num}_bijective_zoomed.npy',np.array(list_bijective_deformed_zoomed_blue))
"""for row in nondeformed_num_zoomed_deformed_to_non_blue:
    if row in nondeformed_num_zoomed_non_to_deformed_blue:
        list_bijective_nondeformed_zoomed_blue.append(row)"""
np.save(f'Bijective Correspondences/{dir_name}/nondeformed_1-{num}_bijective_zoomed.npy',np.array(list_bijective_nondeformed_zoomed_blue))"""


In [ ]:
"""plot_correspondence_lines(mesh1, mesh2, nondeformed, deformed)
print(deformed.shape)
print(nondeformed.shape)"""

In [ ]:
"""print(mesh1.vertices.shape)
print(d.shape)
plot_correspondence_lines(mesh1, mesh2, mesh1.vertices, d)"""